In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
dataset=pd.read_csv(r"C:\Users\aakar\Downloads\Fraud.csv")
dataset.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


# Exploratory Data Analysis

In [3]:
dataset.shape

(6362620, 11)

In [4]:
dataset.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

Therefore, no null values exist in the dataset.

In [5]:
q1 = dataset.quantile(0.25)
q3 = dataset.quantile(0.75)
IQR = q3 - q1

outliers = dataset[((dataset < (q1 - 1.5 * IQR)) | (dataset > (q3 + 1.5 * IQR))).any(axis=1)]

cleaned_dataset = dataset[~((dataset < (q1 - 1.5 * IQR)) | (dataset > (q3 + 1.5 * IQR))).any(axis=1)]

cleaned_dataset.to_csv('Fraud.csv', index=False)

print(cleaned_dataset)

C:\Users\aakar\AppData\Local\Temp\ipykernel_4372\4252901247.py:5: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  outliers = dataset[((dataset < (q1 - 1.5 * IQR)) | (dataset > (q3 + 1.5 * IQR))).any(axis=1)]
C:\Users\aakar\AppData\Local\Temp\ipykernel_4372\4252901247.py:7: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  cleaned_dataset = dataset[~((dataset < (q1 - 1.5 * IQR)) | (dataset > (q3 + 1.5 * IQR))).any(axis=1)]


         step     type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0           1  PAYMENT   9839.64  C1231006815      170136.00       160296.36   
1           1  PAYMENT   1864.28  C1666544295       21249.00        19384.72   
4           1  PAYMENT  11668.14  C2048537720       41554.00        29885.86   
5           1  PAYMENT   7817.71    C90045638       53860.00        46042.29   
6           1  PAYMENT   7107.77   C154988899      183195.00       176087.23   
...       ...      ...       ...          ...            ...             ...   
6259906   601  PAYMENT      3.04   C960748220       25287.00        25283.96   
6259907   601  PAYMENT  13232.49   C737614907       25283.96        12051.47   
6259908   601  PAYMENT   2221.70  C2025555227         496.00            0.00   
6259910   601  PAYMENT   5542.37  C1586821392         465.00            0.00   
6259911   601    DEBIT   4205.96  C1666975703       10856.00         6650.04   

            nameDest  oldbalanceDest  n

In [6]:
cleaned_dataset.shape

(4319406, 11)

### As the while removing outliers it removed arounf approximately 33% data which is a big part so we just move on with original dataset

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [8]:
dataset['type'].value_counts()

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64

As column="Type" is Categorical We need to Convert it to Cantinuous Values

In [9]:
dataset["type"] = dataset["type"].map({"CASH_OUT": 1, "PAYMENT": 2, 
                                 "CASH_IN": 3, "TRANSFER": 4,
                                 "DEBIT": 5})

In [10]:
correlation=dataset.corr()
print(correlation["isFraud"].sort_values(ascending=True))

newbalanceOrig   -0.008148
oldbalanceDest   -0.005885
newbalanceDest    0.000535
oldbalanceOrg     0.010154
type              0.016171
step              0.031578
isFlaggedFraud    0.044109
amount            0.076688
isFraud           1.000000
Name: isFraud, dtype: float64


# Splitting Dataset

In [11]:
X=dataset[["type","amount","oldbalanceOrg", "newbalanceOrig"]]
Y=dataset["isFraud"]

In [12]:
X.head()

,type,amount,oldbalanceOrg,newbalanceOrig
0,2,9839.64,170136.0,160296.36
1,2,1864.28,21249.0,19384.72
2,4,181.00,181.0,0.00
3,1,181.00,181.0,0.00
4,2,11668.14,41554.0,29885.86


In [13]:
Y.head()

0    0
1    0
2    1
3    1
4    0
Name: isFraud, dtype: int64

In [14]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=0.3,stratify = Y, random_state=42)

# Model Training

In [15]:
model = RandomForestClassifier()

In [17]:
model.fit(X_train,Y_train)

RandomForestClassifier()

In [18]:
print("Model Accuracy : ", model.score(X_test,Y_test))

Model Accuracy :  0.9997375295082843


In [20]:
a=int(input("Type (CASH_OUT: 1, PAYMENT: 2, CASH_IN: 3, TRANSFER: 4, DEBIT: 5) :"))
b=float(input("Amount :"))
c=float(input("Old Balance Orig :"))
d=float(input("New Balance Orig :"))
features=np.array([[a,b,c,d]])
if (features == 1).all() :
    print("Fraud")
else :
    print("Not Fraud")

Type (CASH_OUT: 1, PAYMENT: 2, CASH_IN: 3, TRANSFER: 4, DEBIT: 5) :4
Amount :347415.22
Old Balance Orig :14652.00
New Balance Orig :0.00
Not Fraud
